<a href="https://colab.research.google.com/github/Hyeonsik/endotracheal_tube/blob/main/ETT_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download model and sample files

In [1]:
!wget https://github.com/Hyeonsik/endotracheal_tube/raw/main/best_size_model.json
!wget https://github.com/Hyeonsik/endotracheal_tube/raw/main/best_depth_model.json
!wget https://github.com/Hyeonsik/endotracheal_tube/raw/main/sample.csv

--2023-05-28 15:57:12--  https://github.com/Hyeonsik/endotracheal_tube/raw/main/best_size_model.json
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Hyeonsik/endotracheal_tube/main/best_size_model.json [following]
--2023-05-28 15:57:12--  https://raw.githubusercontent.com/Hyeonsik/endotracheal_tube/main/best_size_model.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79664 (78K) [text/plain]
Saving to: ‘best_size_model.json.1’

best_size_model.jso 100%[===================>]  77.80K  --.-KB/s    in 0.002s  

2023-05-28 15:57:12 (46.5 MB/s) - ‘best_size_model.json.1’ saved [79664/79664]

--202

## Endotracheal tube Size prediction

### Sample code for single patient

In [2]:
import xgboost as xgb
import numpy as np


# sample values
### age (years), sex (1 if boy else 0) weight (kg), height (cm), cuffed (1 if cuffed else 0) ###
### y_size (ID; interal diameter(mm)), y_depth (cm) ###
### np.nan for missing value ###
age = 3.28
sex = 1
weight = 17.5
height = 103.2
cuffed = 0
y_size = 5.0
y_depth = 14.0


# Prediction of size (internal diameter of ETT)
## Load the best model
model_size = xgb.XGBRegressor()
model_size.load_model('best_size_model.json')

## Get the predictions
x = np.array([[age, weight, height, cuffed]])
y_pred = model_size.predict(x).flatten()
## Output is rounded up to the nearest size in 0.5 mm
y_pred = np.round(y_pred * 2) / 2

print(f'Used tube size: ID {y_size:.1f} mm,   Predicted tube size: ID {y_pred[0]:.1f} mm')


# Prediction of fixed depth
## Load the best model
model_depth = xgb.XGBRegressor()
model_depth.load_model('best_depth_model.json')

## Get the predictions
x = np.array([[age, weight, height, cuffed]])
y_pred = model_depth.predict(x).flatten()

print(f"Fixed depth: {y_depth:.1f} cm,   Predicted depth: {y_pred[0]:.1f} cm")

Used tube size: ID 5.0 mm,   Predicted tube size: ID 5.0 mm
Fixed depth: 14.0 cm,   Predicted depth: 14.5 cm


### Sample code using CSV file for multiple patients

In [3]:
import xgboost as xgb
import numpy as np
import pandas as pd


df = pd.read_csv('sample.csv', index_col=[0])

# Prediction of size (internal diameter of ETT)
## Load the best model
model_size = xgb.XGBRegressor()
model_size.load_model('best_size_model.json')

INPUT_VARS = ['age_cal','weight','height', 'cuffed']
x = df[INPUT_VARS].astype(float).values

y_pred = model_size.predict(x).flatten()
df['pred_size'] = np.round(y_pred * 2) / 2


# Prediction of fixed depth
## Load the best model
model_depth = xgb.XGBRegressor()
model_depth.load_model('best_depth_model.json')

INPUT_VARS = ['age_cal','sex','weight','height']
x = df[INPUT_VARS].astype(float).values

df['pred_depth'] = model_depth.predict(x).flatten()
df.to_csv('prediction.csv')

print(df)

     age_cal    sex  weight  height  cuffed  size  depth  pred_size  \
1   0.167012   True    5.30    60.0   False   3.5   10.0        3.5   
2   1.144445   True   10.10    78.0   False   4.0   10.5        4.5   
3   0.054758   True     NaN    51.5   False   3.0   10.0        3.5   
4   5.407366  False   18.65   107.4   False   5.0   14.0        5.5   
5   0.221770  False     NaN    58.1   False   3.5   10.0        3.5   
6   7.093917   True   21.60   123.0    True   5.5   16.0        5.5   
7   2.642080   True   13.40    91.0   False   4.5   13.0        5.0   
8   0.375093   True     NaN    67.0   False   4.0   12.5        4.0   
9   3.375839   True   14.70    99.2   False   5.0   14.0        5.0   
10  1.048618   True    8.70    74.0   False   4.5   12.0        4.5   
11  4.210901   True   13.55    98.9   False   5.5   14.0        5.0   
12  5.766032   True   23.55   119.8    True   5.5   15.0        5.5   
13  1.440139   True   13.50    86.0   False   4.0   14.0        4.5   
14  9.